In [15]:
import numpy as np
import nltk
import sys
import operator

In [18]:
UNK = "<unk>"  # token to map all out-of-vocabulary words (OOVs)
UNKid = 0      # index for UNK
epsilon=1e-100
special = "<s>"
smoothing_trans = 1e-2
smoothing_trans_2 = 1e-5

In [198]:
class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                    transition_proba_2 = None,smoothing_obs = 0.01):
            self.N = len(state_list)       # number of states
            self.M = len(observation_list) # number of possible emissions
            #utile pour calcule la proba d un elelement en conaissant la classe observation_list
            
            self.omega_Y = state_list
            self.omega_X = observation_list
            #caractere special
            self.omega_Y.append("@")
            
            if transition_proba is None:
                self.transition_proba = np.zeros( (self.N+1, self.N+1,self.N), float) 
            else:
                self.transition_proba=transition_proba
            #observations
            if observation_proba is None:
                self.observation_proba = np.zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            self.make_indexes() # build indexes, i.e the mapping between token and int
            self.smoothing_obs = smoothing_obs 
            
        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities array"""
            self.Y_index = {}
            for i in range(self.N+1):
                self.Y_index[self.omega_Y[i]] =i
                
            #index des x
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      
        def get_observationIndices( self, observations ):
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices

    
        def data2indices(self, sent): 

            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
            
        def observation_estimation(self, pair_counts):
            # fill with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i = self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        
        def transition_estimation(self, trans_counts,trans_count_bi):
            self.indices_backs = {}
            for pair in trans_counts:
                i=self.Y_index[pair[1]]
                j=self.Y_index[pair[0][0]]
                k=self.Y_index[pair[0][1]]
                self.transition_proba[j,k,i]=float(trans_counts[pair])/float(trans_count_bi[pair[0]])

        def init_estimation(self, init_counts,init_counts2):
            indice_empty = self.Y_index["@"]
            for init in init_counts:
                i=self.Y_index[init]
                self.transition_proba[indice_empty,indice_empty,i]=float(init_counts[init])/float(sum(init_counts.values()))
                
            for p in init_counts2:
                old=self.Y_index[p[0]]
                new=self.Y_index[p[1]]
                self.transition_proba[indice_empty,old,new]=float(init_counts2[p])/float(init_counts[p[0]])
        
        def supervised_training(self, cpairs, ctrans, ctrans2 ,cinits, cinits2):
            """ Train the HMM's parameters. This function wraps everything"""
            self.observation_estimation(cpairs)
            self.transition_estimation(ctrans,ctrans2)
            self.init_estimation(cinits,cinits2)
            #print ctags
            self.solo = np.zeros( (self.N,), float ) 
            for tags in ctags:
                self.solo[self.Y_index[tags]] = ctags[tags]
            self.solo=self.solo/sum(self.solo)
                
        def get_trans(self,k,indi_2):
            #indi_2 : vers nouvel etat
            # k :ancien etat
            return  self.transition_proba[k,indi_2]
            '''index_old = self.Y_index[self.omega_Y_2[k][1]]
            if self.transition_proba[k,indi_2] > 0.0:
                return (1/3.0) * self.transition_proba[k,indi_2] + (1/3.0) * self.transition_proba2[index_old,indi_2]+  (1/3.0)  * self.solo[indi_2]
            index_old = self.Y_index[self.omega_Y_2[k][1]]
            return (1/2.0) *self.transition_proba2[index_old,indi_2] + (1/2.0) * self.solo[indi_2]'''
        
    
        def find_indices(self,k):
            if k == 0 or k==-1:
                return set(['@'])
            return self.omega_Y[0:26]
        def viverbit(self,mots):
            if len(mots) <=1:
                return 0,mots
            alpha = {(0,'@','@') : 1}
            path_to = {('@','@') : []}
            
            
            Y_index_2 = {}
            count = 0 
            for e in self.omega_Y:
                for z in self.omega_Y:
                    Y_index_2[(e,z)]= count
                    count +=1
            
            alpha_2 = np.zeros((self.N+1,self.N+1,len(mots)+1))
            path_to_2 = np.zeros((len(Y_index_2),len(mots)+1))
            
            for j in range(1,len(mots)+1):
                temp_path = {}
                index = 0
                if j==0:
                    index = self.X_index['@']
                else:
                    index = self.X_index[mots[j-1]]
                indices = self.find_indices(j-1)
                for i in indices:
                    corr_indices = self.find_indices(j)
                    indice_i = self.Y_index[i]
                    for co in corr_indices:
                        indice_j= self.Y_index[co]
                        valeur_max = -10000000
                        back = 0
                        for t in self.find_indices(j-2):
                            tmp = alpha[j-1,t,i] * self.transition_proba[self.Y_index[t],indice_i,indice_j] * self.observation_proba[index,indice_j]
                            if tmp > valeur_max:
                                valeur_max = tmp
                                back = t
                        alpha[j,i,co] = valeur_max
                        alpha_2[self.Y_index[i],self.Y_index[co],j] = valeur_max
                        temp_path[i,co] = path_to[back,i] + [co]
                        path_to_2[Y_index_2[(i,co)],j] = self.Y_index[back]
                path_to = temp_path
            #backtrack
            proba = np.max(alpha_2[:,:,len(mots)])
            chemin_max = np.argmax(alpha_2[:,:,len(mots)])
            i,co = Y_index_2.keys()[Y_index_2.values().index(chemin_max)]
            return proba,path_to[i,co]
                 
        def evaluate(self,test_data):
            errors = 0
            total = 0
            erreur_false = 0
            total_false = 0
            erreur_2 = 0
            
            correction = 0
            correction_totale = 0
            correction_ajout = 0
            correction_totale_ajout = 0
            for i in range(len(test_data)):
                if i%100==0:
                    print("Remaining : {}".format(len(test_data)-i))
                p,res = self.viverbit(map(operator.itemgetter(0), test_data[i]))
                if sum([a!=b for a,b in test_data[i]])>0:
                    total_false +=1 
                    erreur_false += sum([a!=b[1] for a,b in zip(res,test_data[i])])
                for a,b in zip(res,test_data[i]):
                    if(b[1]!=b[0] and b[1] ==a):
                        correction +=1
                    if b[1] == '':
                        correction_totale_ajout +=1
                        if a == b[1]:
                            correction_ajout += 1
                    correction_totale +=1
                erreur_2 += sum([a!=b for a,b in test_data[i]])
                errors += sum([a!=b[1] for a,b in zip(res,test_data[i])])
                total += len(res)
            print("################# Resultats du HMM d'ordre 2 #################")
            print("Percentage of errors : {0:.2f}%".format(((errors/float(total))*100.0)))
            print("Taux d erreur brut : {0:.2f}%".format(((erreur_2/float(total))*100.0)))
            print("Taux de correction : {0:.2f}%".format(((correction/float(correction_totale))*100.0)))
            print("Nombre de corrections {} vs nombre de bonnes corrections {}".format(correction_totale,correction))

# Compter les mots et les tags

In [6]:
def make_counts(corpus):
    """ 
    Build different count tables to train a HMM. Each count table is a dictionnary. 
    Returns: 
    * c_words: word counts
    * c_tags: tag counts
    * c_pairs: count of pairs (word,tag)
    * c_transitions: count of tag bigram 
    * c_inits: count of tag found in the first position
    """
    c_words = dict()
    c_tags = dict()
    c_pairs= dict()
    c_transitions = dict()
    c_transitions2 = dict()
    c_inits = dict()
    c_inits2 = dict()
    for sent in corpus:
        # we use i because of the transition counts
        for i in range(len(sent)):
            couple=sent[i]
            wrd = couple[0]
            tag = couple[1]
            # word counts
            if wrd in c_words:
                c_words[wrd]=c_words[wrd]+1
            else:
                c_words[wrd]=1
            # tag counts
            if tag in c_tags:
                c_tags[tag]=c_tags[tag]+1
            else:
                c_tags[tag]=1
            # observation counts
            if couple in c_pairs:
                c_pairs[couple]=c_pairs[couple]+1
            else:
                c_pairs[couple]=1
            # i >  0 -> transition counts
            if i > 0:
                trans = (sent[i-1][1],tag)
                if trans in c_transitions2:
                    c_transitions2[trans]=c_transitions2[trans]+1
                else:
                    c_transitions2[trans]=1
            if i > 1:
                trans = ((sent[i-2][1],sent[i-1][1]),tag)
                if trans in c_transitions:
                    c_transitions[trans]=c_transitions[trans]+1
                else:
                    c_transitions[trans]=1
            # i == 0 -> counts for initial states
            if i==0:
                if tag in c_inits:
                    c_inits[tag]=c_inits[tag]+1
                else:
                    c_inits[tag]=1
            if i == 1:
                cle = (sent[i-1][1],tag)
                if cle in c_inits2:
                    c_inits2[cle] +=1
                else:
                    c_inits2[cle]=1
                    
    return c_words,c_tags,c_pairs, c_transitions, c_inits, c_transitions2,c_inits2

# Création du vocabulaire (filtrage selon le nombre d'occurence)

In [5]:
def make_vocab(c_words, threshold):
    """ 
    return a vocabulary by thresholding word counts. 
    inputs: 
    * c_words : a dictionnary that maps word to its counts
    * threshold: count must be >= to the threshold to be included
    
    returns: 
    * a word list
    """
    voc = list()
    for w in c_words:
        if c_words[w] >= threshold:
            voc.append(w)
    return voc


# les données


In [188]:
import cPickle
from pprint import pprint

with open("typos-data/test10.pkl", "rb") as input_file:
    test = cPickle.load(input_file)
    
with open("typos-data/train10.pkl", "rb") as input_file:
    train = cPickle.load(input_file)
print "Nombre de phrases de train = "+str(len(train))
print "Nombre de phrases de test  = "+str(len(test))

Nombre de phrases de train = 29057
Nombre de phrases de test  = 1501


In [152]:
print train[1]
print test[0]

print test

[('t', 't'), ('h', 'h'), ('e', 'e'), ('i', 'i'), ('r', 'r')]
[('b', 'b'), ('y', 'y')]
[[('b', 'b'), ('y', 'y')]
 [('t', 't'), ('h', 'h'), ('e', 'e'), ('i', 'i'), ('r', 'r')]
 [('o', 'o'), ('w', 'w'), ('n', 'n')] ...,
 [('u', 'u'), ('n', 'n'), ('p', 'p'), ('l', 'l'), ('r', 'e'), ('a', 'a'), ('w', 's'), ('a', 'a'), ('n', 'n'), ('t', 't')]
 [('b', 'b'), ('e', 'e'), ('j', 'h'), ('a', 'a'), ('v', 'v'), ('i', 'i'), ('o', 'o'), ('r', 'r')]
 [('b', 'b'), ('u', 'u'), ('t', 't')]]


In [8]:
cwords,ctags,cpairs,ctrans,cinits, ctrans2,cinits2 = make_counts(train)
print "Nombre de mots  : "+str(len(cwords))
print "Nombre de tags  : "+str(len(ctags))
print "Nombre de paires: "+str(len(cpairs))
print "Nombre de trans : "+str(len(ctrans))+ " / "+ str(12*12)
print "Nombre de init. : "+str(len(cinits))
vocab = make_vocab(cwords,10)
print "Vocabulaire :"+str(len(vocab))

Nombre de mots  : 26
Nombre de tags  : 26
Nombre de paires: 127
Nombre de trans : 2489 / 144
Nombre de init. : 25
Vocabulaire :26


# Modification données pour prendre en compte l'ajout de caractère

In [191]:
import random
from copy import deepcopy

def remove_caracter2(data):
    nb_word_train = len(data)
    nb_modif = int(np.floor(0.1 * nb_word_train))
    word_to_modify = random.sample(range(0, nb_word_train), nb_modif)
    data_modify = deepcopy(data)
    for i in word_to_modify:
        word = deepcopy(data_modify[i])
   
        indice_remove = random.randint(0, len(word)-1)
        
        if indice_remove < len(word)-1 :
            word2 = word[:indice_remove] #+ [('',word[indice_remove][1])] + word[indice_remove+1:]
            #faire une boucle de decalage
            j = indice_remove
            while j < len(word)-1: #s'arrêter à l'avant dernier indice
                word2 = word2 + [(word[j+1][1],word[j][1])]
                j = j + 1
            word2 = word2 + [('',word[j][1])]
        else:   
            word2 = word[:indice_remove] + [('',word[indice_remove][1])]
        data_modify[i] = deepcopy(word2)
    
    # On ajoute un caractère vide à la fin des mots qui n'ont pas été modifié pour ne pas biaiser les tests
    mots_restants = set(range(0,nb_word_train)) - set(list(word_to_modify))
    for j in mots_restants:
        data_modify[j][:] = data_modify[j][:] + [('', '')]
            
    return data_modify

In [192]:
train_modify = deepcopy(remove_caracter2(train))
test_modify = deepcopy(remove_caracter2(test))

In [194]:
print train_modify[11]
print test_modify[1]

[('p', 'l'), ('j', 'i'), ('b', 'b'), ('e', 'e'), ('r', 'r'), ('a', 'a'), ('t', 't'), ('i', 'i'), ('o', 'o'), ('n', 'n'), ('', '')]
[('l', 'l'), ('e', 'e'), ('f', 'f'), ('t', 't'), ('i', 'i'), ('s', 's'), ('t', 't'), ('', '')]


In [195]:
cwords,ctags,cpairs,ctrans,cinits, ctrans2,cinits2 = make_counts(train_modify)
vocab = make_vocab(cwords,10)

In [199]:
hmm = HMM(state_list=ctags.keys(), observation_list=vocab,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None,
                  transition_proba_2 = None,
                 smoothing_obs = 0.00)
hmm.supervised_training( cpairs, ctrans, ctrans2 ,cinits, cinits2)

In [200]:
hmm.evaluate(test_modify)

Remaining : 1501
Remaining : 1401
Remaining : 1301
Remaining : 1201
Remaining : 1101
Remaining : 1001
Remaining : 901
Remaining : 801
Remaining : 701
Remaining : 601
Remaining : 501
Remaining : 401
Remaining : 301
Remaining : 201
Remaining : 101
Remaining : 1
################# Resultats du HMM d'ordre 2 #################
Percentage of errors : 8.52%
Taux d erreur brut : 12.70%
Taux de correction : 5.88%
Nombre de corrections 8671 vs nombre de bonnes corrections 510


In [ ]:

'''
(u'Some', u'DET'), (u'ten', u'NUM'), (u'years', u'NOUN'), (u'ago', u'ADV'), (u'that', u'DET'),
(u'page', u'NOUN'), (u'was', u'VERB'), (u'torn', u'VERB'), (u'out', u'PRT'), (u',', u'.'), 
(u'I', u'PRON'), (u"don't", u'VERB'), (u'know', u'VERB'), (u'by', u'ADP'), (u'whom', u'PRON'), (u'.', u'.')]
'''


'''
Projet : variable n est plus mot mais caractere
observer un mot => sequence d etats la version correcte
Xik = convluwion
Yik = conclusion

pour que ce soit plus dur => modele de second ordre

donnees (bon char, mauvais char)

 deux version : 10 : 10% d erreurs
                 20 : 20%  d erreurs
                 
3 points : hmm et biterdi : taux d erreurs du modele

modele markov ordre deux : => proba  d un etat depend des deux etats precedents P(Yt|Yt-1,Yt2)

dans viberti : on propage (dans delta) la proba du meilleur chemin pour arriver a un instant t dans un etat donne
=> quand rempli t on peut remplir t+1 (par hypothese) markov d ordre 1 => donc a ordre 2 il faut faire changement de varibale
besoin de garder la trace de tout ce qui est contionnement (Yt garde trace de ce qui est en t-1 et t-2 )
taille devient s² => delta de taille (K*N^2)

3 novembre : finir partie 1
'''